# 数据载入

In [1]:
import pandas as pd
import os
from collections import Counter
import numpy as np

import torch

In [2]:
pd.set_option('display.max_colwidth', -1)

C:\Users\c'y'y\AppData\Local\Temp\ipykernel_7252\3652881321.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [26]:
poi_path =r'E:\xpj\research\POI\psqlcsv\chennai.csv'
poi_prompt_path =r'E:\xpj\research\POI\psqlcsv\chennai_poi_prompt.csv'
image_path = r'E:\xpj\research\POI\psqlcsv\chennai_img_location.csv'
image_poi_path = r'E:\xpj\research\POI\psqlcsv\chennai_img_poi.csv'
desc_path =r'E:\xpj\research\POI\desc\chennai_desc.csv'

## poi数据（全面）

In [5]:
poi_data = pd.read_csv(poi_path,encoding='utf-8')
# poi_data = pd.read_csv(r"E:\xpj\research\POI\mumbai_poi.csv",encoding='ANSI')
poi_df = pd.DataFrame(poi_data)

In [6]:
poi_df

,id,geom,osm_id,name,barrier,highway,ref,address,is_in,place,man_made,other_tags
0,1,0101000020E6100000D4DE967EF81054407D0F4D8F5C292A40,30037235,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0101000020E6100000B3785677D1105440E00E79701C292A40,30037236,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,NaN
2,3,0101000020E6100000C8844EBE8F1054406D95BB2AAB282A40,30037239,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0101000020E6100000E09D7C7A6C1054404DE5A37B8C282A40,30037240,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,NaN
4,5,0101000020E61000009D1A1EB14C10544078EE3D5C72282A40,30037241,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
18810,18811,0101000020E6100000B4B0A71D7E0854405B16026F371B2A40,11380759873,NaN,gate,NaN,NaN,NaN,NaN,NaN,NaN,"""access""=>""private"""
18811,18812,0101000020E6100000A884CC3A080C5440A8774C384F182A40,11380806603,NaN,gate,NaN,NaN,NaN,NaN,NaN,NaN,"""access""=>""private"""
18812,18813,0101000020E61000005DB41199030C5440267156444D182A40,11380806604,NaN,gate,NaN,NaN,NaN,NaN,NaN,NaN,"""access""=>""private"""
18813,18814,0101000020E610000035ED629A690B544009A0BD9FBF192A40,11380819223,NaN,gate,NaN,NaN,NaN,NaN,NaN,NaN,"""access""=>""private"""


In [7]:
poi_df.columns

Index(['id', 'geom', 'osm_id', 'name', 'barrier', 'highway', 'ref', 'address',
       'is_in', 'place', 'man_made', 'other_tags'],
      dtype='object')

In [8]:
# save tag list to txt

poi_df_nec=poi_df.loc[:,['id', 'name', 'barrier', 'highway','ref', 'address', 'place', 'man_made', 'other_tags']]
poi_df_ot =poi_df_nec.loc[:,['other_tags']]
tag_list = []
def collect_tags(x):
    # print(x['other_tags'])
    
    if  pd.notna(x['other_tags']):
        # print(x['other_tags'])
        tags = x['other_tags'].split("\",\"")
        # print(tags)
        for tag in tags:
            try:
                tag = tag.replace("\"","")
                t = tag.split("=>")[0]
                if t not in tag_list:
                    tag_list.append(t)
            except:
                pass
    # return tag_list

result = poi_df_ot.apply(lambda x: collect_tags(x), axis=1, result_type='expand')
# result
tag_list.sort()
tag_list
# 保存tag_list到txt
with open(r'E:\xpj\research\POI\chennai_tag_list.txt', 'w') as f:
    for item in tag_list:
        f.write("%s\n" % item)

# poi_df_ot

# def genSent(x):
#     tags = None
#     b = True    
#     # sen = "The imformation of this point contains that "
#     sen = "Nearby,"
#     for key in ['name','barrier', 'highway','address','place','man_made']:
#         if not x[key].isna().iloc[0]:
#             # print(x[key])
#             sen = sen + key +" is " + x[key].iloc[0] + ","
#     if not x['other_tags'].isna().iloc[0]:
#         tags = x['other_tags'].iloc[0].split("\",\"")
#         for tag in tags:
#             try:
#                 tag = tag.replace("\"","")
#                 t = tag.split("=>")[0]
#                 k = tag.split("=>")[1]
#                 if "name:" in t and t != "name:en":
#                     pass
#                 sen = sen + t +" is " + k + ","
#             except:
#                 pass

#     sen = sen + "located in " + x['NAME_2'].iloc[0]
    
#     # return sen,b
    
#     return sen
# # poi_df_nec


## 照片信息

In [9]:
pic_data = pd.read_csv(image_path,encoding='utf-8')
# pic_data = pd.read_csv(r"E:\xpj\research\POI\mumbai_points.csv",encoding='ANSI')
pic_df = pd.DataFrame(pic_data)

In [10]:
pic_df

,id,geom,captured_a,camera_mak,created_at,sequences_,pano,user_key,username,img_key,ca,longitude,latitude,img_id,lon,lat,layer,path
0,1,0101000020E6100000FD0C141D6E085440CED83CB3C9222A40,2018/12/09 05:51:22.000,Xiaomi,2018/12/17 09:09:54.000,8eL92EWBT_WY7SneBXFnZQ,0,Z49rqRyqnx9-E8klWnJcPg,gnuanwar,HITGwLx46CMXc12qPBcfCw,0.0000,80.131721,13.067945,NaN,NaN,NaN,Chennai_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/Chennai_img_location.shp
1,2,0101000020E6100000833BAB60AF0E5440F23F9E0205132A40,2018/11/13 12:47:27.000,Xiaomi,2018/12/17 09:18:47.000,gAYiPW9PRYaXuiLgL2wFxA,0,Z49rqRyqnx9-E8klWnJcPg,gnuanwar,GwLzPvp47wn1zjdOnlu-bw,98.3636,80.229454,13.037148,NaN,NaN,NaN,Chennai_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/Chennai_img_location.shp
2,3,0101000020E6100000A2AFC513E6115440BB0F9B2386302A40,2015/12/27 13:21:24.000,ASUS,2015/12/27 18:48:12.000,jsQdYiQeBFiQeMToPvUYjg,0,bhbwfMdubPMiem62NcMRow,jnikheel,cMqkD1UWIzurHBFd9vxlew,19.2491,80.279668,13.094773,NaN,NaN,NaN,Chennai_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/Chennai_img_location.shp
3,4,0101000020E61000009E4143FF84115440ECF593D684212A40,2018/11/13 08:09:08.000,Xiaomi,2018/12/17 09:19:36.000,Cw9mOmLsSEuMWVeo8nO26A,0,Z49rqRyqnx9-E8klWnJcPg,gnuanwar,n0oi7dgXMrFlx88Thdk90g,135.5690,80.273742,13.065467,NaN,NaN,NaN,Chennai_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/Chennai_img_location.shp
4,5,0101000020E610000055FB743CE611544031348DDC89302A40,2015/12/27 13:21:24.000,ASUS,2015/12/27 18:48:12.000,jsQdYiQeBFiQeMToPvUYjg,0,bhbwfMdubPMiem62NcMRow,jnikheel,opVh1wE4jauRMmseM5XONw,19.3060,80.279678,13.094802,NaN,NaN,NaN,Chennai_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/Chennai_img_location.shp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65431,65432,0101000020E61000006B2BF697DD0F5440029A081B9E1E2A40,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,qJkMIhRptD2J9dSv0KL33Q,80.2479,13.0598,view_location_chennai,C:/Users/24491/Documents/FileRecv/region/google_view_locations/view_location_chennai.shp
65432,65433,0101000020E61000006B2BF697DD0F54401F85EB51B81E2A40,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,tdl6sk6ju60QToJErQwl_w,80.2479,13.0600,view_location_chennai,C:/Users/24491/Documents/FileRecv/region/google_view_locations/view_location_chennai.shp
65433,65434,0101000020E61000006B2BF697DD0F54403A92CB7F481F2A40,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,g79a4uJHTjuZwasSHulPHg,80.2479,13.0611,view_location_chennai,C:/Users/24491/Documents/FileRecv/region/google_view_locations/view_location_chennai.shp
65434,65435,0101000020E61000006B2BF697DD0F5440E5F21FD26F1F2A40,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,7UlpLRP_xoQKKNA-GVHheQ,80.2479,13.0614,view_location_chennai,C:/Users/24491/Documents/FileRecv/region/google_view_locations/view_location_chennai.shp


## 照片poi关联

In [11]:
pic_poi_data = pd.read_csv(image_poi_path,encoding='utf-8')
# pic_poi_data = pd.read_csv(r"E:\xpj\research\POI\mumbai_pois_to_points.csv",encoding='ANSI')
pic_poi_df = pd.DataFrame(pic_poi_data)

In [12]:
pic_poi_df

,img_key,id1,distance1,id2,distance2,id3,distance3,id4,distance4,id5,distance5
0,006FVFVo79NVF5mU0KEEEQ,5350,111.963349,11307.0,153.090632,463.0,193.448932,10484.0,193.936430,9672.0,198.380046
1,00bSGYdpzQLKnkt-pjx9mQ,5354,23.679344,10464.0,44.556633,5620.0,60.159711,576.0,61.288758,14569.0,64.984850
2,00G7KbVnC5n9k12CcmZ4Zq,480,283.860742,12572.0,284.366458,NaN,NaN,NaN,NaN,NaN,NaN
3,00GszrzsspUZi4VuniOWFw,6624,13.160351,10222.0,13.807160,17760.0,16.607754,6631.0,16.632122,10225.0,22.146077
4,00RmeRDxXEkugAY__dSkdw,5085,28.855696,9195.0,29.701039,2049.0,35.445383,9725.0,40.351135,11385.0,54.355617
...,...,...,...,...,...,...,...,...,...,...,...
31779,ZZRY6_REQ-Y2x4PPINvT2A,7589,341.202166,50.0,346.081939,12401.0,459.683906,12403.0,460.613950,12402.0,461.815276
31780,ZZTLO9FTVJ0N74fRzv7jIQ,11241,63.450452,11243.0,104.324831,11251.0,125.443542,1841.0,214.685140,11195.0,235.510223
31781,ZZVXwqtEYbz0wOtsFj3iqh,9837,46.492950,14921.0,74.044216,7843.0,103.120122,2307.0,106.008084,12639.0,106.220007
31782,zZW76PkRuJipT2beMSds1n,2059,26.522135,262.0,50.539175,2775.0,63.741657,2884.0,207.751140,2883.0,211.880039


## 照片描述

In [14]:
desc_pd =pd.read_csv(desc_path,encoding='utf-8')
desc_pd

,image,prompt
0,-08wICnojXYYB0UP-wNh5w_80.2859_13.0888.jpg,"arafed building with a broken window and a man walking by, a detailed matte painting by Rajesh Soni, featured on pexels, neoplasticism, dilapidated houses, photo of poor condition, dilapidated look"
1,-1Xpqvci1MHqVT1dlSrtaw_80.2889_13.0885.jpg,"cars are driving down a busy street in a city, a matte painting by Thota Vaikuntham, polycount, hurufiyya, city street view background, garden road, found on google street view"
2,-8TlGk_Tc4blyhP41tdi1w_80.2856_13.0795.jpg,"arafed building with a car parked in front of it, a detailed matte painting by Thota Vaikuntham, behance, bengal school of art, found on google street view, google maps street view, google street view"
3,-9PsvvlQHtDDNEplmrKnTg_80.2792_13.0834.jpg,"cars parked on the side of the road near a building, a tilt shift photo by Thota Vaikuntham, polycount, samikshavad, found on google street view, bangalore, trading depots"
4,-AnxAzIZLshju3mG26L4Sg_80.287_13.0789.jpg,"arafed image of a parking lot with a man in uniform, a detailed matte painting by Thota Vaikuntham, polycount, bengal school of art, found on google street view, google point of view, google maps street view"
...,...,...
24230,_ZsdX-JL8uKtASLpmLWHvQ_80.2483_13.1128.jpg,"arafed street scene with a man standing in front of a store, a detailed matte painting by Thota Vaikuntham, polycount, samikshavad, found on google street view, lots of shops, google maps street view"
24231,__h2iVUOUZDqvPHPIu8o9A_80.255_13.1125.jpg,"arafed building with a balcony and a balcony on the second floor, a detailed matte painting by Thota Vaikuntham, behance, samikshavad, dilapidated house, dilapidated houses, dilapidated look"
24232,__NH7rpsEXTc80Y_kJePTQ_80.2616_13.1263.jpg,"there are two people standing outside of a building with a bike, a microscopic photo by Ram Chandra Shukla, polycount, samikshavad, found on google street view, google maps street view, google street view"
24233,__NILdSUQnnvVy-G2u4Z0w_80.2451_13.1222.jpg,"there is a pile of junk in front of a building, a detailed matte painting inspired by Riza Abbasi, dribbble, environmental art, exposed wires, earthquake destruction, electrical plant location"


# 数据处理

## poi数据预处理

In [15]:
poi_df_nec=poi_df.loc[:,['id', 'name', 'barrier', 'highway', 'address', 'place', 'man_made', 'other_tags']]

In [16]:
poi_df_nec

,id,name,barrier,highway,address,place,man_made,other_tags
0,1,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN
1,2,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN
2,3,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN
3,4,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN
4,5,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
18810,18811,NaN,gate,NaN,NaN,NaN,NaN,"""access""=>""private"""
18811,18812,NaN,gate,NaN,NaN,NaN,NaN,"""access""=>""private"""
18812,18813,NaN,gate,NaN,NaN,NaN,NaN,"""access""=>""private"""
18813,18814,NaN,gate,NaN,NaN,NaN,NaN,"""access""=>""private"""


In [ ]:
# poi_df_nec.to_csv(r"E:\xpj\research\POI\poi_df_nec.csv")

# 硬提示计算相似度，找最佳poi

In [17]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# 加载BERT模型和分词器
tokenizer = AutoTokenizer.from_pretrained(r"E:\xpj\models\bert-base-uncased")
model = AutoModel.from_pretrained(r"E:\xpj\models\bert-base-uncased")

In [18]:
# 定义计算相似度的函数
def calc_similarity(s1, s2):
    # 对句子进行分词，并添加特殊标记
    inputs = tokenizer([s1, s2], return_tensors='pt', padding=True, truncation=True)
    # print(inputs)

    # 将输入传递给BERT模型，并获取输出
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()

    # 计算余弦相似度，并返回结果
    sim = np.dot(embeddings[0], embeddings[1]) / (np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1]))
    return sim


In [20]:

def isEnglish(s):
    if isinstance(s, str):
        return s.isascii()
    return False



# 从txt中读取tag_list
tag_ignore_list = []
with open(r'E:\xpj\research\POI\tag_ignore_list.txt', 'r') as f:
    for line in f.readlines():
        tag_ignore_list.append(line.strip())
# print(tag_ignore_list)

def genSent(x):
    tags = None
    b = True    
    # sen = "The imformation of this point contains that "
    sen = "Nearby,"
    for key in ['name','barrier', 'highway','address','place','man_made']:
        # if isinstance(x[key],str) and not isEnglish(x[key]):
        #     continue
        if key == 'name' and not isEnglish(x['name'].iloc[0]):
            continue
        if not x[key].isna().iloc[0]:
            # print(x[key])
            sen = sen + key +" is " + x[key].iloc[0] + ","
    # print(sen)
    if not x['other_tags'].isna().iloc[0]:
        tags = x['other_tags'].iloc[0].split("\",\"")
        for tag in tags:
            try:
                tag = tag.replace("\"","")
                t = tag.split("=>")[0]
                k = tag.split("=>")[1]
                
                if not isEnglish(k) or not isEnglish(t):
                    # print(k,t)
                    continue
                if list(t)[-1] == "=":
                    continue
                if t in tag_ignore_list:
                    continue
                # if "name:" in t and t != "name:en":
                #     pass
                # pass 的作用是什么也不做 跳过，continue的作用是执行下次for循环
                # print(t,k)
                sen = sen + t +" is " + k + ","
            except:
                pass

    # sen = sen + "located in " + x['NAME_2'].iloc[0]
    
    # return sen,b
    
    return sen

In [ ]:
# a="脿陇陇脿陇戮脿陇聹脿陇庐脿陇鹿脿陇戮脿陇脿陇陋脿楼聟脿陇虏脿楼聡脿陇脿陇鹿脿楼聣脿陇聼脿楼聡脿陇"
# # print(isEnglish(a))
# b="à¤­à¤¾à¤°à¤¤à¥à¤à¤¹à¤µà¤¾à¤®à¤¾à¤¨ à¤µà¤¿à¤­à¤¾à¤"
# print(isEnglish(b))
# print(b.encode('utf-8').decode('ISO-8859-1'))
s = "啶ぞ啶班い啷€啶啶灌さ啶距ぎ啶距え 啶掂た啶ぞ啶"
print(s.encode('ansi').decode('iscii'))

将poi对应的自然语言提示存储到文件中(poi_nec最后插入一列提示)

In [21]:
def store_poi(x):
    # print(x)
    poi = x[0]
    # print(poi)    
    poi_im = poi_df_nec.loc[poi_df_nec['id']==poi]
    sen = genSent(poi_im)
    return sen
    # print(sen)


    

In [22]:
from tqdm import tqdm
result =[]
for index, row in tqdm(poi_df_nec.iterrows(), total=poi_df_nec.shape[0], desc="Processing"):
    # if index == 130:
    #     print(store_poi(row))
    result.append(store_poi(row))
# result = poi_df_nec.apply(lambda x: store_poi(x), axis=1, result_type='expand')
# poi_df_nec['poi_prompt']= result
# poi_df_nec['poi_prompt'] = result

Processing: 100%|██████████| 18815/18815 [00:21<00:00, 873.83it/s]


In [27]:
# poi_df_nec.to_csv(r"E:\xpj\research\POI\poi_df_nec3.csv")
poi_df_nec['poi_prompt'] = result
poi_df_nec.to_csv(poi_prompt_path)
poi_df_nec

,id,name,barrier,highway,address,place,man_made,other_tags,poi_prompt
0,1,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,"Nearby,highway is traffic_signals,"
1,2,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,"Nearby,highway is traffic_signals,"
2,3,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,"Nearby,highway is traffic_signals,"
3,4,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,"Nearby,highway is traffic_signals,"
4,5,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,"Nearby,highway is traffic_signals,"
...,...,...,...,...,...,...,...,...,...
18810,18811,NaN,gate,NaN,NaN,NaN,NaN,"""access""=>""private""","Nearby,barrier is gate,"
18811,18812,NaN,gate,NaN,NaN,NaN,NaN,"""access""=>""private""","Nearby,barrier is gate,"
18812,18813,NaN,gate,NaN,NaN,NaN,NaN,"""access""=>""private""","Nearby,barrier is gate,"
18813,18814,NaN,gate,NaN,NaN,NaN,NaN,"""access""=>""private""","Nearby,barrier is gate,"


In [28]:
def find_best_poi(x):
    # print(x)
    img_key = x['img_key']
    img_file = img_key+'.jpg'
    try:
        img_desc = desc_pd.loc[desc_pd['image']==img_file]['prompt'].iloc[0]
    except:
        print("The description of picture " + img_file +" doesn't exist")
        return  None,None
    pois = x[1::2]
    pois_dis =x[2::2]
    # print(pois)
    similarities = []
    sentences = []
    for index,poi in enumerate(pois):
        # 设置距离阈值
        if pois_dis[index]<100:
            poi_im = poi_df_nec.loc[poi_df_nec['FID_points']==poi]
            sen = genSent(poi_im)
        
            similarity = calc_similarity(img_desc,sen)
            sentences.append(sen)
            similarities.append(similarity)
        
    if len(similarities) == 0:
        best_poi=None
        return best_poi,0
    elif similarities and max(similarities)>0.5:
        best_poi = pois[similarities.index(max(similarities))]
        return best_poi,max(similarities)

In [30]:
result = []
for _, row in tqdm(pic_poi_df.iterrows(), total=pic_poi_df.shape[0], desc="Processing"):
    result.append(find_best_poi(row))

pic_poi_df[['best_poi', 'best_poi_score']] = result

Processing:   0%|          | 0/31784 [00:00<?, ?it/s]

The description of picture 006FVFVo79NVF5mU0KEEEQ.jpg doesn't exist


Processing:   0%|          | 1/31784 [11:30<6099:04:51, 690.83s/it]

The description of picture 00bSGYdpzQLKnkt-pjx9mQ.jpg doesn't exist
The description of picture 00G7KbVnC5n9k12CcmZ4Zq.jpg doesn't exist


Processing:   0%|          | 111/31784 [11:31<31:42:09,  3.60s/it] 

...

The description of picture ZlYzbfKQeUayJ454WsgpJg.jpg doesn't exist
The description of picture ZLZWpDeTMWOPnIi5E5LY5A.jpg doesn't exist
The description of picture zM7H4RMl7wjd9GuxuqxPiQ.jpg doesn't exist
The description of picture zm7MfbxShmzqtIM7KdO975.jpg doesn't exist
The description of picture ZM7TItlLux7bbOrsMJZvsw.jpg doesn't exist
The description of picture Zm_9rektJ9UxQlyt2XET_w.jpg doesn't exist
The description of picture ZmCNWJJnFfK7BmHLsTknRQ.jpg doesn't exist
The description of picture zmdjyyZWtQiz1BlP2aYGYQ.jpg doesn't exist
The description of picture zMe2cshFREBvKucydOMsQQ.jpg doesn't exist
The description of picture zMeHdtt__aoOKaRJCNWhPQ.jpg doesn't exist
The description of picture zMeMmG4YrmcFkNuhWWbtjA.jpg doesn't exist
The description of picture ZMEtSl2RhkNSTD8R90nJr8.jpg doesn't exist
The description of picture zMhokXPIuvVDY7L4Bw41F2.jpg doesn't exist
The description of picture zmIE9GH5DS4xrkFVPqVtQj.jpg doesn't exist
The description of picture zMISrdvgxMLjwnB9USqX-

Processing: 100%|█████████▉| 31645/31784 [12:24<00:00, 592.21it/s]

The description of picture Zshne3pGzBgeE8mPq8WvDA.jpg doesn't exist
The description of picture zs-ioiCjnxNxZNa4bolnNA.jpg doesn't exist
The description of picture zSJmZJvnpOxWK0tL3SwZTv.jpg doesn't exist
The description of picture zSKDU0tI0abk1114kSM1Wh.jpg doesn't exist
The description of picture zskO9EZB9baGB6fi4WK6cg.jpg doesn't exist
The description of picture ZSkPoTITcUPoX3nm01m_Aw.jpg doesn't exist
The description of picture zSLaENtFvxlvpqHnOPvNpg.jpg doesn't exist
The description of picture ZslESOwx5N9QoEZiqMm4N9.jpg doesn't exist
The description of picture zSN0iFWwWoBrzAaSkyK2Bk.jpg doesn't exist
The description of picture ZsORR6dsceSrObYpRtRQTh.jpg doesn't exist
The description of picture zSpmi0FnlTaFIUs15nxYSx.jpg doesn't exist
The description of picture ZsQhJPdDX4vSswCNFwTBYw.jpg doesn't exist
The description of picture zSRA1g0dRVmGSuJaN38zNA.jpg doesn't exist
The description of picture zsuID79KeZ4WIPdA0gsH0B.jpg doesn't exist
The description of picture ZSvfRuw4uoSs00ObGCV6Z

Processing: 100%|██████████| 31784/31784 [12:24<00:00, 42.69it/s] 

The description of picture Zvm29lpzy-IrbKBUBIS02w.jpg doesn't exist
The description of picture zvMkbSuFXYZ4LrEWTzO3po.jpg doesn't exist
The description of picture zVNOoIxL40mGkQpdSfDwdw.jpg doesn't exist
The description of picture ZVPOSJiWyoha31RpQcCiPL.jpg doesn't exist
The description of picture ZVQBtZ40GX6Z10Y797tqW1.jpg doesn't exist
The description of picture ZVrUfwPmdqK_rXKGH2YCXQ.jpg doesn't exist
The description of picture zvrVG_vx-TTNxolKmT16lg.jpg doesn't exist
The description of picture ZvSgHnoHHGG2htIWD0qgVZ.jpg doesn't exist
The description of picture zvu4Cp5oDZhAM_Z25f-Qew.jpg doesn't exist
The description of picture Zvvgo1nJKTigiLMdqfLgRQ.jpg doesn't exist
The description of picture ZvVI3NBlh6oZD4XrYvnnjT.jpg doesn't exist
The description of picture zVvkS9Ol5qm1s2b5HI1G8A.jpg doesn't exist
The description of picture zvvxO93mKLhyTLVOELZtJn.jpg doesn't exist
The description of picture zvW4cTEEilmotJOhZxWs8v.jpg doesn't exist
The description of picture zVWz2e7iTg5rbqnNUwWOv

In [ ]:
result = pic_poi_df.apply(lambda x: find_best_poi(x), axis=1,result_type='expand')

In [ ]:
pic_poi_df[['best_poi','similarity']]= result

In [ ]:
pic_poi_df

In [ ]:
pic_poi_df.to_csv(r"E:\xpj\research\POI\\best_poi2.csv")

In [ ]:
df1 = pd.merge(pic_poi_df, poi_df_nec, left_on="best_poi",right_on = 'FID_points',how="left")

In [ ]:
df1

In [ ]:
df1.to_csv(r"E:\xpj\research\POI\\best_poi_information2.csv")

In [ ]:
df1 = pd.read_csv(r"E:\xpj\research\POI\best_poi_information2.csv")
poi_df_nec.columns

In [ ]:
# desc_pd: file,prompt insert poi_prompt
# df1 : image_key, bestpoi
# poi_df_nec: FID ,poi_prompt
def connect(x):
    image_file_name = x[0]
    image_name =image_file_name.split('.')[0]
    best_poi_row = df1.loc[df1['img_key']==image_name]['best_poi']
    best_poi = None
    if best_poi_row.notna().any():
        best_poi=int(best_poi_row.iloc[0])
        poi_prompt = poi_df_nec.loc[poi_df_nec['FID_points']==best_poi]['poi_prompt'].iloc[0]
        return poi_prompt
        # print(poi_prompt)
    # print(best_poi)
    # print("\n")
    

    # print(image_name)

In [ ]:
result2 = desc_pd.apply(lambda x: connect(x), axis=1,result_type='expand')
result2

In [ ]:
desc_pd['poi_prompt'] = result2

In [ ]:
desc_pd

In [ ]:
desc_pd.to_csv(r"E:\xpj\research\POI\desc_poi2.csv")